**Authors** - Daniel Cohen, ID: 213059714 ; Gal Elhiani, ID: 207233248

**AI-Driven Discord Server Management Bot**

Description:

This project focuses on developing an AI-powered bot to manage Discord servers using natural language commands. Discord is a popular platform for communities, but server management often requires technical knowledge or familiarity with structured commands. The proposed solution bridges this gap by integrating natural language processing (NLP) with the Discord API to create a bot capable of interpreting and executing user requests in plain English.

The bot will handle tasks such as creating text or voice channels, managing role permissions, and organizing server settings. For example, a user could input, "Create a voice channel visible only to members with the 'friends' role," and the bot would parse the request and perform the action autonomously. This involves designing an NLP pipeline to extract intent and entities from user input and mapping them to API functions.

The project combines deep learning models for NLP with rule-based parsing to balance accuracy and efficiency. It also incorporates robust error handling to address invalid or ambiguous inputs. The result is a highly accessible and user-friendly tool for server administrators.

By applying deep learning to practical API interaction, this project demonstrates the potential of AI in simplifying complex workflows and improving user experience in digital platforms.

In [145]:
!pip install discord
!pip install python-dotenv

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
responses_path = '/content/drive/MyDrive/Discord_Manager_files/responses.ipynb'

In [148]:
import os
if os.path.exists(responses_path):
    print("Script found!")
else:
    print("Script not found. Check the path.")

Script found!


In [149]:
import sys
folder_path = '/content/drive/MyDrive/Discord_Manager_files'
sys.path.append(folder_path)


In [150]:
!cp /content/drive/MyDrive/Discord_Manager_files/responses.ipynb /content

In [151]:
!cp /content/drive/MyDrive/Discord_Manager_files/.env /content

It crucial to run the responses.ipynb in order to use the model, otherwise it won't work.

In [152]:
import os
from dotenv import load_dotenv
from discord import Intents, Client, Message, Guild, utils
import asyncio
%run /content/drive/MyDrive/Discord_Manager_files/responses.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load Discord key

In [153]:
load_dotenv()
TOKEN = os.getenv('DISCORD_KEY')

Set up the bot

In [154]:
intents = Intents.default()
intents.message_content = True
intents.members = True
client = Client(intents=intents)

Message functionality
async and await:

async: Used to define a coroutine. A coroutine is a special kind of function that can pause its execution and yield control back to the event loop, allowing other tasks to run.

await: Used to pause the execution of a coroutine until the awaited task (usually another coroutine or an asynchronous operation) completes. While waiting, the event loop can switch to other tasks.

In [155]:
async def send_message(message: Message, user_message: str, users_list, roles_list):
  if not user_message:
    raise ValueError("Message is empty, intents were not enabled properly")
  if not (user_message[0] == '!'): # In order to talk to the bot you must first use "!" in the beginning of the input
    return

  user_message = user_message[1:]
  response = get_response(user_message, users_list, roles_list)
  yes_no = " (yes/no)"
  output_text = f"{response}{yes_no}"
  try:
    await message.channel.send(output_text)
    def check(m):
            return m.author == message.author and m.channel == message.channel

    # Wait for the message with a timeout of 30 seconds
    follow_up = await client.wait_for('message', check=check, timeout=30)

    # Process the follow-up message
    if follow_up.content.lower() == 'yes':
        await message.channel.send(f"Action confirmed for {message.author.name}!")
        await execute_action(message.guild, response)  # Execute the action
        await message.channel.send(f"Action Done for {message.author.name}!")
    elif follow_up.content.lower() == 'no':
        await message.channel.send(f"Action cancelled for {message.author.name}.")
    else:
        await message.channel.send(f"Invalid response. Please retype the message and answer 'yes' or 'no'.")

  except asyncio.TimeoutError:
      await message.channel.send(f"{message.author.name}, you took too long to respond. Action cancelled.")


Translate the prediction output into executable actions

In [156]:

async def execute_action(guild, action_dict):
    action = action_dict.get('action')
    user_name = action_dict.get('user')
    role_name = action_dict.get('role')

    # Find the actual user and role in the guild (=Server)
    user = utils.get(guild.members, name=user_name)
    role = utils.get(guild.roles, name=role_name)

    if not user:
        await guild.system_channel.send(f"User '{user_name}' not found in the server!")
        return
    if role_name != "null" and not role:
        await guild.system_channel.send(f"Role '{role_name}' not found in the server!")
        return

    # Perform the appropriate action
    if action == "ban":
        await user.ban()
        await guild.system_channel.send(f"Banned {user.name}.")
    elif action == "add_role" and role:
        await user.add_roles(role)
        await guild.system_channel.send(f"Added role '{role.name}' to {user.name}.")
    elif action == "remove_role" and role:
        await user.remove_roles(role)
        await guild.system_channel.send(f"Removed role '{role.name}' from {user.name}.")
    else:
        await guild.system_channel.send("Unknown action received!")




Start up the Bot

In [157]:
@client.event
async def on_ready():
  print(f'{client.user} is now running!')

Handling messages

In [158]:
@client.event
async def on_message(message):
  if message.author == client.user:
    return
  guild = message.guild
  users_list = [member.name for member in guild.members]
  roles_list = [role.name for role in guild.roles]
  username = str(message.author)
  user_message = str(message.content)
  channel = str(message.channel)
  if(channel == 'dog-testing'):
    print(f'[{channel}] {username}: "{user_message}"')
    await send_message(message, user_message, users_list, roles_list)

Creating main function

In [159]:
async def main():
  await client.start(token=TOKEN)

if __name__ == '__main__':
  await main()

Dog_Manager#4933 is now running!
[dog-testing] bambi1826: "!"


<ipython-input-152-e6cf7929697c>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
ERROR:disc

[dog-testing] bambi1826: "!1"
[dog-testing] bambi1826: "no"
[dog-testing] bambi1826: "!lets add a role for yoav31, i want him to be a chess player"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!i want you to remove the role of chess player from yoav31"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!add role bambi2106vs chess player"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!lets add a role to bambi2106vs i want him to become a chess player"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!i want you to remove bambi2106vs role of chess player"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!bambi2106 is insane give him an admin"
[dog-testing] bambi1826: "!hey there i want you to add bambi2106vs Admin he deserves it"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!bambi2106vs has been abusing his power, revoke his Admin privilleges"
[dog-testing] bambi1826: "yes"
[dog-testing] bambi1826: "!ban bambi2106vs he is spamming"

CancelledError: 